# Bayes by Backprop: Mushroom Contextual Bandit

__Background - Multi-armed and Contextual Bandit problems:__ <br>
_“Bandit”_ in _“multi-armed bandits”_ comes from _“one-armed bandit”_ machines used in a casino. Imagine that you are in a casino with many one-armed bandit machines. Each machine has a different probability of a win. Your goal is to maximize total payout. You can pull a limited number of arms, and you don’t know which bandit to use to get the best payout. The problem involves exploration/exploitation tradeoff: you should balance between trying different bandits to learn more about an expected payout of every machine, but you also want to exploit the best bandit you know about more. While multi-armed bandits select the strategy which maximises the expected reward without taking into consideration the state of the environment, _contextual bandits_ output an action based on the context.

__Mushroom Case Stuty:__ <br>
We are provided with a list of 8124 mushrooms, each having 22 features (characteristcs of the mushroom) and 1 label (poisonous or edible). Our agent can carry out 2 actions: eat a mushroom or not eat a mushroom. The problem context is the vector of features which is associated with the mushroom which the agent is about to eat/non eat.
If our agent eats an edible mushroom, it receives a reward of 5. If the agent eats a poisonous mushroom, it receives a reward of -35 with 0.5 probability and a reward of 5 with 0.5 probaility. If the agent doesn't eat, it receives a reward o 0.

We are also provided an _oracle_. The oracle always selects the right action, receiving a reward of 5 when it eats an edible mushroom, and a reward of 0 when it doesn't eat.

__Objective:__ <br>
Create a BNN which minimises the _cumulative regret_ of the agent. Regreat measures the difference between the oracle and our agent's reward.

__BNN Architecture:__
- 2 hidden layers
- Each layer has 100 rectified units
- Vector input: vector consisting of the mushroom features (context) and a one of _K_ encoding of the action
- Output: single scalar representing the expected reward of the given action in the given context

__Additional Important Information:__
- To calculate the expected reward for an action, Google samples twice the weights and averages the outputs
- To train the network, Google keeps a buffer of 4096 mushrooms.
- For training, Google randomly draws minibatches of size 64 for 64 training steps.
- After every training stage, the agent interacts with a new mushroom



In [12]:
# Import Libraries

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import preprocessing
from sklearn import metrics
import random
import sys
sys.path.append('../../')
from BayesBackpropagation import *
import warnings
warnings.filterwarnings('ignore')

## __Data Preparation:__ ## 
Input is the data file. Output is an array of one-hot encoded labels (y, shape: 8124) and an array of context features (x, shape: 8124 elements, each with 117 features). 117 features derive from doing a one-hot encoding on the feature classes.

In [13]:
# Import data from file
df = pd.read_csv(os.getcwd() + '/agaricus-lepiota.data', sep=',', header=None,
             error_bad_lines=False, warn_bad_lines=True, low_memory=False)

# Set pandas to output all of the columns in output
df.columns = ['class','cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment',
         'gill-spacing','gill-size','gill-color','stalk-shape','stalk-root',
         'stalk-surf-above-ring','stalk-surf-below-ring','stalk-color-above-ring','stalk-color-below-ring',
         'veil-type','veil-color','ring-number','ring-type','spore-color','population','habitat']

# Split context from label
X = pd.DataFrame(df, columns=df.columns[1:len(df.columns)], index=df.index)
# Put the class values (0th column) into Y
Y = df['class']

# Transform labels into one-hot encoded array
le = preprocessing.LabelEncoder()
le.fit(Y)
y = le.transform(Y)

# Temporary variable to avoid error 
x_tmp = pd.DataFrame(X,columns=[X.columns[0]])

# Encode each feature column and add it to x_train 
for colname in X.columns:
    le.fit(X[colname])
    x_tmp[colname] = le.transform(X[colname])

# Produce mushroom array: 8124 mushrooms, each with 117 one-hot encoded features
oh = preprocessing.OneHotEncoder(categorical_features='all')
oh.fit(x_tmp)
x = oh.transform(x_tmp).toarray()


## __Reward function:__ ##
Implemented as described above

In [14]:
x.shape

(8124, 117)

In [15]:
# REWARD FUNCTION

def get_reward(eaten, edible):
    # REWARDS FOR AGENT
    #  Eat poisonous mushroom
    if not eaten:
        return 0
    if eaten and edible:
        return 5
    elif eaten and not edible:
        return (5 if np.random.rand() > 0.5 else -35)

def oracle_reward(edible):
    return 5*edible    

## Create Mushroom Buffer
Generate buffer to keep 4096 elements used to train the Bayesian Neural Network

## Neural Network


In [16]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
print("Cuda available?: ",torch.cuda.is_available())

PI = 0.75
SIGMA_1 = torch.FloatTensor([math.exp(-1)])
SIGMA_2 = torch.FloatTensor([math.exp(-6)])

Cuda available?:  False


### Create class for Gaussian distribution

In [17]:
import torch.nn as nn
import torch.nn.functional as F
import sys

Var = lambda x, dtype=torch.FloatTensor: Variable(torch.from_numpy(x).type(dtype))


class MushroomNet():
    def __init__(self, label = 'MushNet', n_weight_sampling=2):
        self.label = label
        self.n_weight_sampling = n_weight_sampling
        self.epsilon = 0
        self.net = None
        self.loss, self.optimizer = None, None
        self.cum_regrets = [0]
        self.bufferX = []
        self.bufferY = []
        
    # Use NN to decide next action    
    def try_ (self, mushroom):
        samples = self.n_weight_sampling
        context, edible = x[mushroom], y[mushroom]
        try_eat = Var(np.concatenate((context, [1, 0])))
        try_reject = Var(np.concatenate((context, [0, 1])))
        
        # Calculate rewards using model
        with torch.no_grad():
            r_eat = np.mean([self.net.forward(try_eat).numpy() for _ in range(samples)])
            r_reject = np.mean([self.net.forward(try_reject).numpy() for _ in range(samples)])
                         
        # Take random action for epsilon greedy agents, calculate agent's reward
        eaten = r_eat > r_reject
        if np.random.rand()<self.epsilon:
            eaten = (np.random.rand()<.5)
        agent_reward = get_reward(eaten, edible)
        
        # Get rewards and update buffer
        if eaten:
            action = [1, 0]
        else:
            action = [0, 1]
        self.bufferX.append(np.concatenate((context, action)))
        self.bufferY.append(agent_reward)
    
        # Calculate regret
        oracle = oracle_reward(edible)
        regret = oracle - agent_reward
        self.cum_regrets.append(self.cum_regrets[-1]+regret) 
    
        
    # Function for generating a minibatch
    def generate_minibatch(self, trainingX, trainingY):
        bX = []
        bY = []
        for i in range(64):
            random = np.random.randint(0, len(trainingX))
            bX.append(trainingX[random])
            bY.append(trainingY[random])
        return bX, bY
    
    # Feed next mushroom
    def update(self, mushroom):
        self.try_(mushroom)
        for minibatch in range(64):
            bX, bY = self.generate_minibatch(self.bufferX[-4096:], self.bufferY[-4096:])
            self.net.zero_grad()
            self.loss(Var(np.asarray(bX)), Var(np.asarray(bY))).backward()
            self.optimizer.step()
            

In [18]:
# Class for BBB agent
class BBB_MNet(MushroomNet):
    def __init__(self, label, lr=0, **kwargs):
        super().__init__(label, **kwargs)
        self.net = BayesianNetwork(inputSize = x.shape[1]+2,
                          CLASSES = 1, 
                          layers=np.array([100,100]), 
                          activations = np.array(['relu','relu','none']), 
                          SAMPLES = 2, 
                          BATCH_SIZE = 64,
                          NUM_BATCHES = 64,
                          hasScalarMixturePrior=True,
                          PI=PI,
                          SIGMA_1 = SIGMA_1,
                          SIGMA_2 = SIGMA_2,
                          GOOGLE_INIT=True,
                          ).to(DEVICE)
        self.lr = lr
        self.optimizer = optim.Adam(self.net.parameters(), lr = lr)
        self.loss = lambda data, target:self.net.BBB_loss(data, target)
        
        
# Class for Greedy agents
class EpsGreedyMlp(MushroomNet):
    def __init__(self, epsilon=0, lr=0, **kwargs):
        super().__init__(**kwargs)
        self.n_weight_sampling = 1
        self.epsilon = epsilon
        self.lr = lr
        self.net = nn.Sequential(
        nn.Linear(x.shape[1]+2, 100), nn.ReLU(),
        nn.Linear(100, 100), nn.ReLU(),
        nn.Linear(100, 1))
        self.optimizer = optim.SGD(self.net.parameters(), lr = lr)
        self.mse = nn.MSELoss(reduction = 'sum')
        self.loss = lambda data, target: self.mse(self.net.forward(data), target)        

In [19]:
mushroom_nets = {
                 'bbb0.00001':BBB_MNet(label = 'BBB 0.00001', lr = 0.001),
                 
                 'e0.00001':EpsGreedyMlp(epsilon=0, label = 'Greedy 0.00001', lr = 0.00001),
                 
                 'e1.00001':EpsGreedyMlp(epsilon=0.01, label = '1% Greedy 0.00001', lr = 0.00001),

                 'e5.00001':EpsGreedyMlp(epsilon=0.05, label = '5% Greedy 0.00001', lr = 0.00001),}

In [20]:
import plotly.graph_objs as go
class Fig(go.FigureWidget):
    def __init__(self):
        super().__init__([go.Scatter(name = net.label, x=[], y=[]) 
                          for i, net in mushroom_nets.items()])
        self.layout.title = 'Cumulative Regrets'
    
    def update(self, dic):
        for j, (lb, y) in enumerate(dic.items()):
            self.data[j].x = list(range(len(y)))
            self.data[j].y = y
    
    def save(self, file_name = 'mushroom_regrets.csv'):
        dic = {fd.name: fd.y for fd in self.data}
        pd.DataFrame.from_dict(dic).to_csv(file_name)
        
        
fig = Fig()
fig  

Fig({
    'data': [{'name': 'BBB 0.00001', 'type': 'scatter', 'uid': 'c9df0eed-9edb-4d41-bc49-7faa19b5cd41', 'x': [], 'y': []},
             {'name': 'Greedy 0.00001',
              'type': 'scatter',
              'uid': '1bd9ad79-d490-4001-82ef-3bcef35a370b',
              'x': [],
              'y': []},
             {'name': '1% Greedy 0.00001',
              'type': 'scatter',
              'uid': '4386ab2e-f5d0-4590-b026-9073a2e1a4ac',
              'x': [],
              'y': []},
             {'name': '5% Greedy 0.00001',
              'type': 'scatter',
              'uid': '3ed977d6-7fa0-4cc6-aabc-8894adc26758',
              'x': [],
              'y': []}],
    'layout': {'title': {'text': 'Cumulative Regrets'}}
})

In [21]:
import plotly.offline as py
py.iplot(fig)

In [ ]:
NB_STEPS = 10000


offset = len(fig.data[0].x)
print([net.label for lb, net in mushroom_nets.items()])

for _ in range(offset, offset+NB_STEPS):
    mushroom = np.random.randint(0, len(x))
    for j, (key, net) in enumerate(mushroom_nets.items()):
        net.update(mushroom)
    fig.update({net.label: net.cum_regrets for lb, net in mushroom_nets.items()})
    if _ % 10 == 0 :
        print('Step',_,'Regrets',[net.cum_regrets[-1] for lb, net in mushroom_nets.items()])

['BBB 0.00001', 'Greedy 0.00001', '1% Greedy 0.00001', '5% Greedy 0.00001']
Step 0 Regrets [5, 5, 5, 0]
Step 10 Regrets [55, 65, 10, 30]
Step 20 Regrets [95, 70, 50, 20]
Step 30 Regrets [155, 105, 85, 85]
Step 40 Regrets [205, 185, 95, 155]
Step 50 Regrets [215, 205, 100, 300]
Step 60 Regrets [225, 215, 140, 300]
Step 70 Regrets [220, 240, 180, 310]
Step 80 Regrets [265, 260, 185, 320]
Step 90 Regrets [330, 285, 190, 325]
Step 100 Regrets [340, 335, 230, 365]
Step 110 Regrets [385, 385, 270, 395]
Step 120 Regrets [390, 405, 275, 430]
Step 130 Regrets [395, 425, 275, 440]
Step 140 Regrets [475, 450, 275, 440]
Step 150 Regrets [485, 495, 310, 430]
Step 160 Regrets [560, 515, 310, 425]
Step 170 Regrets [565, 550, 310, 425]
Step 180 Regrets [590, 585, 315, 435]
Step 190 Regrets [635, 605, 325, 440]
Step 200 Regrets [655, 640, 370, 440]
Step 210 Regrets [690, 655, 375, 440]
Step 220 Regrets [690, 675, 380, 475]
Step 230 Regrets [700, 700, 415, 470]
Step 240 Regrets [695, 720, 415, 465]
Step

Step 1980 Regrets [2455, 5595, 1715, 2560]
Step 1990 Regrets [2500, 5615, 1715, 2550]
Step 2000 Regrets [2520, 5630, 1720, 2585]
Step 2010 Regrets [2530, 5660, 1730, 2625]
Step 2020 Regrets [2530, 5670, 1730, 2625]
Step 2030 Regrets [2545, 5700, 1730, 2620]
Step 2040 Regrets [2550, 5715, 1730, 2655]
Step 2050 Regrets [2565, 5735, 1770, 2660]
Step 2060 Regrets [2565, 5745, 1770, 2660]
Step 2070 Regrets [2580, 5765, 1770, 2660]
Step 2080 Regrets [2590, 5775, 1770, 2655]
Step 2090 Regrets [2605, 5790, 1805, 2655]
Step 2100 Regrets [2620, 5820, 1840, 2655]
Step 2110 Regrets [2625, 5835, 1875, 2650]
Step 2120 Regrets [2635, 5865, 1875, 2655]
Step 2130 Regrets [2640, 5885, 1915, 2655]
Step 2140 Regrets [2680, 5890, 1915, 2725]
Step 2150 Regrets [2695, 5920, 1915, 2760]
Step 2160 Regrets [2700, 5940, 1915, 2795]
Step 2170 Regrets [2715, 5955, 1915, 2865]
Step 2180 Regrets [2725, 5975, 1915, 2865]
Step 2190 Regrets [2725, 5980, 1920, 2895]
Step 2200 Regrets [2730, 5990, 1920, 2895]
Step 2210 R

Step 3890 Regrets [3670, 9615, 2565, 4930]
Step 3900 Regrets [3670, 9630, 2605, 4995]
Step 3910 Regrets [3665, 9650, 2610, 5030]
Step 3920 Regrets [3665, 9670, 2610, 5070]
Step 3930 Regrets [3665, 9690, 2610, 5075]
Step 3940 Regrets [3665, 9785, 2610, 5075]
Step 3950 Regrets [3665, 9805, 2610, 5070]
Step 3960 Regrets [3665, 9820, 2610, 5070]
Step 3970 Regrets [3735, 9845, 2610, 5070]
Step 3980 Regrets [3735, 9870, 2610, 5070]
Step 3990 Regrets [3735, 9900, 2610, 5070]
Step 4000 Regrets [3735, 9925, 2645, 5100]
Step 4010 Regrets [3730, 9945, 2640, 5095]
Step 4020 Regrets [3730, 9975, 2640, 5130]
Step 4030 Regrets [3730, 10000, 2640, 5130]
Step 4040 Regrets [3730, 10015, 2640, 5125]
Step 4050 Regrets [3735, 10035, 2645, 5130]
Step 4060 Regrets [3730, 10060, 2645, 5160]
Step 4070 Regrets [3730, 10075, 2645, 5190]
Step 4080 Regrets [3765, 10090, 2645, 5190]
Step 4090 Regrets [3765, 10105, 2680, 5190]
Step 4100 Regrets [3770, 10125, 2685, 5225]
Step 4110 Regrets [3770, 10155, 2685, 5220]
St